# 9. Python Standart Kitabxanası

---


### 9.5 Pathlib - Pythonda fayl sistemi yolları ilə işləməyin müasir yolu

1. Bir path alaraq, onun haqqında məlumatları çap edən `get_info` funksiyasını yazın. Aşağıdakıları çap etməyə çalışın:
    - Yolun tipi - fayl ya qovluq olması
    - həcmi - MB, KB və sairə
    - Yaradılma tarixi
    - Son dəyişmə tarixi

In [3]:
from pathlib import Path
from datetime import datetime

def get_info(path: Path):
    if not path.exists():
        raise FileNotFoundError(f"{path} does not exist!")
    
    path_type = "file" if path.is_file() else "directory"
    stats = path.stat()
    return f":: {path}:\n" + \
           f"\tTYPE: {path_type}\n" + \
           f"\tSIZE: {stats.st_size} bytes\n" + \
           f"\tCREATED: {datetime.fromtimestamp(stats.st_ctime)}\n" + \
           f"\tMODIFIED: {datetime.fromtimestamp(stats.st_mtime)}"

print(get_info(Path.home() / "Documents" / 'code'))

:: /home/tengo/Documents/code:
	TYPE: directory
	SIZE: 4096 bytes
	CREATED: 2025-12-29 18:47:26.502206
	MODIFIED: 2025-12-29 18:47:26.502206


---

2. qovluğun yolunu alıb, içərisindəki elementləri nömrələnmiş və "gözəl" şəkildə format edilmiş halda çap edəcək `ls` funksiyasını yazın.

In [ ]:
from pathlib import Path

# həcmi lazımi vahidlə göstərməkçün yardımçı funksiya
def format_size(bytes_size):
    """Convert bytes to human-readable format"""
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if bytes_size < 1024.0:
            return f"{bytes_size:6.1f} {unit}"
        bytes_size /= 1024.0
    return f"{bytes_size:6.1f} PB"

def ls(path: Path):
    """ list files and directories in `path` """
    if not path.is_dir():
        raise ValueError(f"'{path}' must be a directory!")
    
    items = list(path.iterdir())
    
    # Print header
    print(f"\n📁 {path}")
    print("─" * 75)
    print(f"{'#':<4} {'Name':<32} {'Size':<12} {'Modified'}")
    print("─" * 75)
    
    # Sort: directories first, then files (alphabetically)
    items.sort(key=lambda x: (not x.is_dir(), x.name.lower()))
    
    for i, item in enumerate(items, start=1):
        postfix = '/' if item.is_dir() else ''
        name = item.name + postfix
        
        # Truncate long names
        if len(name) > 30:
            name = name[:27] + "..."
        
        size = format_size(item.stat().st_size)
        modified = datetime.fromtimestamp(item.stat().st_mtime).strftime("%Y-%m-%d %H:%M")
        
        print(f"{i:<4} {name:<32} {size:<12} {modified}")
    
    print("─" * 75)
    print(f"Total: {len(items)} items ({sum(1 for i in items if i.is_dir())} dirs, {sum(1 for i in items if i.is_file())} files)\n")

ls(Path.home() / "Documents" / 'code')


---

3. Öncəki `get_info` və `ls` funksiyalarından istifadə edərək, bir `file_browser` funksiyasını yazın. Bu funksiya ilə, ötürülmüş yoldakı faylların siyahısına baxmaq(`ls`), sıra nömrəsinə görə seçilmiş qovluğa keçib içərisinə baxmaq olar. Seçilən qovluqdursa, içərisindəkiləri `ls` edirik, fayldırsa - `get_info` ilə məlumatları çap edirik. İstifadəçi hər zaman "b" daxil edərək valideyn qovluğuna keçə, "q" daxil edərək menyunu tərk edə və ya cari nömrələnmiş(əgər qovluqda `ls` nəticəsinə baxırsa) siyahıdan elementin sıra nömrəsini daxil edə bilər.

In [ ]:
from pathlib import Path
from datetime import datetime

def format_size(bytes_size):
    """Convert bytes to human-readable format"""
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if bytes_size < 1024.0:
            return f"{bytes_size:6.1f} {unit}"
        bytes_size /= 1024.0
    return f"{bytes_size:6.1f} PB"

def get_info(path: Path):
    if not path.exists():
        raise FileNotFoundError(f"{path} does not exist!")
    
    path_type = "file" if path.is_file() else "directory"
    stats = path.stat()
    return f":: {path}:\n" + \
           f"\tTYPE: {path_type}\n" + \
           f"\tSIZE: {format_size(stats.st_size)}\n" + \
           f"\tCREATED: {datetime.fromtimestamp(stats.st_ctime)}\n" + \
           f"\tMODIFIED: {datetime.fromtimestamp(stats.st_mtime)}"

def ls(path: Path)->list[Path]:
    """ list files and directories in `path` """
    if not path.is_dir():
        raise ValueError(f"'{path}' must be a directory!")
    
    items = list(path.iterdir())
    
    print(f"\n📁 {path}")
    print("─" * 75)
    print(f"{'#':<4} {'Name':<32} {'Size':<12} {'Modified'}")
    print("─" * 75)
    
    # əlifba sırası ilə sıralayaq, əvvəlcə qovluqlar, sonra isə fayllar
    items.sort(key=lambda x: (not x.is_dir(), x.name.lower()))
    
    for i, item in enumerate(items, start=1):
        postfix = '/' if item.is_dir() else ''
        name = item.name + postfix
        
        # çox uzun faylların sonlarını ataq
        if len(name) > 30:
            name = name[:27] + "..."
        
        size = format_size(item.stat().st_size)
        modified = datetime.fromtimestamp(item.stat().st_mtime).strftime("%Y-%m-%d %H:%M")
        
        print(f"{i:<4} {name:<32} {size:<12} {modified}")
    
    print("─" * 75)
    print(f"Total: {len(items)} items ({sum(1 for i in items if i.is_dir())} dirs, {sum(1 for i in items if i.is_file())} files)\n")
    
    return items

def file_browser(start_path: Path):
    """
    Interactive file browser.
    - Enter a number to navigate into a folder or view file info
    - Enter 'b' to go back to parent directory
    - Enter 'q' to quit
    """
    current_path = start_path.resolve()
    
    while True:
        # əsas dövr
        items = ls(current_path)
        
        choice = input("Enter number to select, 'b' for back, 'q' to quit: ").strip().lower()
        
        if choice == 'q':
            print("👋 Exiting file browser...")
            break
        
        elif choice == 'b':
            # valideynə keçid edirik
            parent = current_path.parent
            if parent != current_path:
                current_path = parent
            else:    # geri-geri gedib ən əsas kök(root) qovluğa çatsaq
                print("⚠️  Already at root directory!")
        
        else:
            # istifadəçi yazan ədədi oxuyuruq
            try:
                index = int(choice) - 1  # siyahı 1-dən başladığı üçün indeks kimi istifadə etməkçün -1 edirik
                
                if 0 <= index < len(items):
                    selected_item = items[index]
                    
                    if selected_item.is_dir():
                        # qovluğa keçirik
                        current_path = selected_item
                    else:
                        # fayl haqqında məlumatları göstəririk.
                        print()
                        print(get_info(selected_item))
                        print()
                        input("Press Enter to continue...")
                else:
                    print(f"⚠️  Invalid number! Please enter a number between 1 and {len(items)}")
            
            except ValueError:
                print("⚠️  Invalid input! Enter a number, 'b', or 'q'")


file_browser(Path.home() / "Documents" / "code")


📁 /home/tengo/Documents/code
───────────────────────────────────────────────────────────────────────────
#    Name                             Size         Modified
───────────────────────────────────────────────────────────────────────────
1    bicycles/                           4.0 KB    2025-07-31 23:07
2    icma/                               4.0 KB    2025-07-31 23:04
3    icma-v/                             4.0 KB    2025-07-31 23:08
4    managR/                             4.0 KB    2025-08-09 19:00
5    nightingale/                        4.0 KB    2025-07-31 23:14
6    open-tech/                          4.0 KB    2026-01-02 20:10
7    open-tech-lessons/                  4.0 KB    2025-10-16 17:08
8    ParVu/                              4.0 KB    2026-01-05 23:30
9    phoner/                             4.0 KB    2025-08-01 07:42
10   realt_az/                           4.0 KB    2025-12-29 16:11
11   rustic/                             4.0 KB    2025-07-31 23:07
12   smart


📁 /home/tengo/Documents/code/ParVu
───────────────────────────────────────────────────────────────────────────
#    Name                             Size         Modified
───────────────────────────────────────────────────────────────────────────
1    .claude/                            4.0 KB    2025-12-30 20:40
2    .git/                               4.0 KB    2026-01-05 23:29
3    .venv/                              4.0 KB    2026-01-05 23:30
4    assets/                             4.0 KB    2025-12-30 20:31
5    build/                              4.0 KB    2025-12-30 20:44
6    dist/                               4.0 KB    2025-12-30 20:45
7    docs/                               4.0 KB    2025-12-30 20:18
8    src/                                4.0 KB    2025-12-30 20:57
9    .gitignore                          2.5 KB    2025-12-29 23:06
10   .python-version                     5.0 B     2025-12-29 18:50
11   build.ps1                           9.6 KB    2025-12-30 00:31
12  

---